In [22]:
import pandas as pd 
import requests
data = requests.get('https://m.trademap.org/#/grid/391620/076/0/export/country', verify=False)
BRAZIL_CODE = '076'
SH6_CODE = "391620"

/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [24]:
import requests

url = "https://m.trademap.org/api/ByCountry"

payload = "{\"language_code\":\"en\",\"trade_type\":\"I\",\"product_type\":6,\"product_code\":\"391620\",\"sort_expression\":\"value desc\"}"
headers = {
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'en-US,en;q=0.6',
  'Connection': 'keep-alive',
  'Content-Type': 'application/json;charset=UTF-8',
  'Origin': 'https://m.trademap.org',
  'Referer': 'https://m.trademap.org/',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-GPC': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

top_importers = pd.DataFrame.from_dict(response.json())
top_importers.head()


/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,country_code,country_label,unit,value,trade_balance,quantity,unit_value,trend_value,trend_value_2a,trend_quantity,share_world,distance,concentration,tariff
0,000,World,No quantity,3795057.0,275194.0,0.0,NaN,7.0,34.0,NaN,100.0,1922.0,0.17,NaN
1,842,United States of America,Tons,449083.0,-339986.0,134747.0,3333.0,13.0,37.0,9.0,11.8,2637.0,0.67,2.2
2,616,Poland,Tons,267894.0,135050.0,88117.0,3040.0,13.0,29.0,7.0,7.1,681.0,0.81,1.7
3,276,Germany,Tons,260539.0,1116367.0,82733.0,3149.0,9.0,38.0,4.0,6.9,829.0,0.27,1.7
4,642,Romania,Tons,201709.0,-179521.0,81296.0,2481.0,9.0,44.0,5.0,5.3,1121.0,0.29,1.7


In [29]:
url = "https://m.trademap.org/api/BySpecificCountry"
payload = "{\"language_code\":\"en\",\"trade_type\":\"E\",\"product_type\":6,\"product_code\":\"391620\",\"country_code\":\"076\",\"country_reporting\":\"D\",\"sort_expression\":\"value desc\"}"
headers = {
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'en-US,en;q=0.6',
  'Connection': 'keep-alive',
  'Content-Type': 'application/json;charset=UTF-8',
  'Origin': 'https://m.trademap.org',
  'Referer': 'https://m.trademap.org/',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-GPC': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
brazil_exports = pd.DataFrame.from_dict(response.json())
brazil_exports = brazil_exports[['partner_code','value']]
brazil_exports.head()

/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,partner_code,value
0,000,24067.0
1,858,5558.0
2,032,5171.0
3,600,4185.0
4,068,2174.0


In [48]:
final_df = top_importers[['country_code', 'country_label', 'value']]
final_df = pd.merge(final_df, brazil_exports, left_on='country_code', right_on='partner_code', how='left')
final_df = final_df[1:11] # we only want top 10 importers
final_df

,country_code,country_label,value_x,partner_code,value_y
1,842,United States of America,449083.0,842,5.0
2,616,Poland,267894.0,616,NaN
3,276,Germany,260539.0,276,44.0
4,642,Romania,201709.0,642,NaN
5,251,France,198938.0,251,NaN
6,381,Italy,165114.0,381,NaN
7,528,Netherlands,141354.0,528,NaN
8,203,Czech Republic,114047.0,203,NaN
9,368,Iraq,112268.0,368,NaN
10,124,Canada,94498.0,124,NaN


In [40]:
def get_single_importers_data(country_code):
    url = "https://m.trademap.org/api/BySpecificCountry"

    payload = f"{{\"language_code\":\"en\",\"trade_type\":\"I\",\"product_type\":6,\"product_code\":\"391620\",\"country_code\":\"{country_code}\",\"country_reporting\":\"D\",\"sort_expression\":\"value desc\"}}"
    headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.6',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'https://m.trademap.org',
    'Referer': 'https://m.trademap.org/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-GPC': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
    }

    return requests.request("POST", url, headers=headers, data=payload, verify=False)

In [50]:
response = get_single_importers_data(842)

/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [51]:
response.json()

[{'reporter_code': '000',
  'partner_code': '000',
  'partner_label': 'World',
  'value': 449083.0,
  'trade_balance': -339986.0,
  'share': 100.0,
  'quantity': 134747.0,
  'quantity_unit_label': 'Tons',
  'unit_value': 3333.0,
  'trend_value': 13.0,
  'trend_quantity': 9.0,
  'trend_value_2a': 37.0,
  'tariff': None,
  'share_partners': 100.0,
  'distance': 2637.0,
  'concentration': 0.67},
 {'reporter_code': '842',
  'partner_code': '124',
  'partner_label': 'Canada',
  'value': 364164.0,
  'trade_balance': -290472.0,
  'share': 81.1,
  'quantity': 107096.0,
  'quantity_unit_label': 'Tons',
  'unit_value': 3400.0,
  'trend_value': 12.0,
  'trend_quantity': 9.0,
  'trend_value_2a': 39.0,
  'tariff': 0.0,
  'share_partners': 9.1,
  'distance': 1189.0,
  'concentration': 0.98},
 {'reporter_code': '842',
  'partner_code': '156',
  'partner_label': 'China',
  'value': 36568.0,
  'trade_balance': -36339.0,
  'share': 8.1,
  'quantity': 15002.0,
  'quantity_unit_label': 'Tons',
  'unit_val

In [52]:
single_importer = pd.DataFrame.from_dict(response.json())[1:]
single_importer['partner_label'][1]

'Canada'

In [53]:
final_df

,country_code,country_label,value_x,partner_code,value_y
1,842,United States of America,449083.0,842,5.0
2,616,Poland,267894.0,616,NaN
3,276,Germany,260539.0,276,44.0
4,642,Romania,201709.0,642,NaN
5,251,France,198938.0,251,NaN
6,381,Italy,165114.0,381,NaN
7,528,Netherlands,141354.0,528,NaN
8,203,Czech Republic,114047.0,203,NaN
9,368,Iraq,112268.0,368,NaN
10,124,Canada,94498.0,124,NaN


In [54]:
for country_code in final_df['country_code']:
    try:
        response = get_single_importers_data(country_code)
        single_importer = pd.DataFrame.from_dict(response.json())[1:]

        single_importer['position'] = single_importer.reset_index().index

        first_position_name = single_importer['partner_label'][1]
        second_position_name = single_importer['partner_label'][2]
        third_position_name = single_importer['partner_label'][3]

        first_position_pct = str(single_importer['share'][1]).replace('.',',')
        second_position_pct = str(single_importer['share'][2]).replace('.',',')
        third_position_pct = str(single_importer['share'][3]).replace('.',',')

        position_string = f"{first_position_name}/{first_position_pct}%, {second_position_name}/{second_position_pct}%, {third_position_name}/{third_position_pct}%"
        
        
        final_df.loc[final_df['country_code'] == country_code, 'position_string'] = position_string
    except:
        pass
    print(country_code)

/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


842


/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


616


/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

203


/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


368


/home/rbercini/miniconda3/envs/exporta_rs/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.trademap.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


124


In [55]:
final_df.loc[final_df['country_code'] == country_code, 'position_string'] = position_string

In [56]:
single_importer['position'] = single_importer.reset_index().index

In [57]:
final_df

,country_code,country_label,value_x,partner_code,value_y,position_string
1,842,United States of America,449083.0,842,5.0,"Canada/81,1%, China/8,1%, Mexico/2,5%"
2,616,Poland,267894.0,616,NaN,"Germany/90,1%, Belgium/2,7%, Türkiye/1,2%"
3,276,Germany,260539.0,276,44.0,"Poland/49,6%, Netherlands/11,7%, Austria/7,6%"
4,642,Romania,201709.0,642,NaN,"Germany/47,0%, Poland/22,7%, Türkiye/13,1%"
5,251,France,198938.0,251,NaN,"Germany/51,7%, Belgium/19,0%, Poland/11,1%"
6,381,Italy,165114.0,381,NaN,"Germany/71,4%, Poland/7,6%, France/6,8%"
7,528,Netherlands,141354.0,528,NaN,"Germany/67,8%, Belgium/22,9%, United Kingdom/2,1%"
8,203,Czech Republic,114047.0,203,NaN,"Germany/77,1%, Poland/11,8%, Italy/2,1%"
9,368,Iraq,112268.0,368,NaN,NaN
10,124,Canada,94498.0,124,NaN,"United States of America/70,5%, China/17,0%, G..."


In [58]:
final_df.loc[final_df['country_code'] == '842', 'position_string'] = position_string